In [1]:
import re
import googletrans
import jaydebeapi

import pandas as pd 
import numpy as np 

# 문자열 중복제거
from collections import OrderedDict

#  google 번역
import googletrans

# db 연결 
import pymysql 
import jaydebeapi


import pickle
import time 


# python google translate 라이브러리
translator = googletrans.Translator()


In [2]:
# DB connection
conn = jaydebeapi.connect(
    "com.tmax.tibero.jdbc.TbDriver",
    "jdbc:tibero:thin:@172.7.0.23:8629:tibero",
    ["labelon", "euclid!@)$labelon"],
    "tibero6-jdbc.jar",
    )
cur = conn.cursor()

In [3]:
# pandas  불러오기 

all_select_sql = 'select * from TE_DATA_BASIC_INFO'
df = pd.read_sql_query(all_select_sql, conn )
df

C:\Users\yoonsub\.conda\envs\project01\lib\site-packages\pandas\io\sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


,ID,COLLECT_SITE_ID,CATEGORY_BIG,CATEGORY_SMALL,DATA_NAME,DATA_DESCRIPTION,KEYWORDS,CREATOR_ORGANIZATION,CREATOR_DEPARTMENT,CREATOR_TELEPHONE,...,COLLECT_DATA_TYPE,COLLECT_URL_LINK,SAVE_FILE_PATH,SAVE_FILE_NAME,SAVE_FILE_EXTENTION,IS_COLLECT_YN,SYSTEM_REGISTER_DATE,SYSTEM_UPDATED_DATE,DATA_ORIGIN_KEY,FIX_API_URL
0,609,2,공공데이터,공공행정,인천광역시 남동구_도서관대출현황,"인천광역시 남동구 도서관대출현황에 대한 데이터로 연번, 도서관명, 구분, 도서, 비...",None,None,None,None,...,[File],[https://www.data.go.kr/data/15086970/fileData...,None,None,None,N,None,None,15086970,None
1,610,2,공공데이터,사회복지,사립학교교직원연금공단_지역별 퇴직 교직원 수,"사립학교교직원연금공단 지역별 퇴직 교직원 수와 관련된 데이터로 지역별, 학교급별(유...",None,None,None,None,...,[File],[https://www.data.go.kr/data/15048976/fileData...,None,None,None,N,None,None,15048976,None
2,611,2,공공데이터,사회복지,사립학교교직원연금공단_지역별 가입 교직원 수,"사립학교교직원연금공단 지역별 가입 교직원 수와 관련된 데이터로 지역별, 학교급별(유...",None,None,None,None,...,[File],[https://www.data.go.kr/data/15048977/fileData...,None,None,None,N,None,None,15048977,None
3,612,2,공공데이터,공공행정,서울특별시 동작구_장애인직업재활시설,이 데이터는 서울특별시 동작구 관내에 있는 장애인직업재활시설에 관한 것입니다. 재활...,None,None,None,None,...,[File],[https://www.data.go.kr/data/15037433/fileData...,None,None,None,N,None,None,15037433,None
4,613,2,공공데이터,사회복지,사립학교교직원연금공단_재직기간별 학교급별 교직원 퇴직 현황,"사립학교교직원연금공단 재직기간별 학교급별 교직원 퇴직 현황과 관련된 데이터로 연도,...",None,None,None,None,...,[File],[https://www.data.go.kr/data/15045810/fileData...,None,None,None,N,None,None,15045810,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21005,24824,2,공공데이터,과학기술,기상청_식중독지수,기상청에서 발표한 생활기상지수 중 식중독지수 자료입니다.,None,None,None,None,...,[File],[https://www.data.go.kr/data/15043581/fileData...,None,None,None,N,None,None,15043581,None
21006,24825,2,공공데이터,과학기술,기상청_라디오미터 관측자료,센서를 통해 하양 마이크로파 복사값을 관측하고 구름탐지를 위해 적외창에서의 하양복사...,None,None,None,None,...,[File],[https://www.data.go.kr/data/15043606/fileData...,None,None,None,N,None,None,15043606,None
21007,24826,2,공공데이터,과학기술,기상청_표류부이경로,"기상청에서 관측하는 표류부이의 이동경로로서 지점,관측시간,위도,경도 등을 제공합니다.",None,None,None,None,...,[File],[https://www.data.go.kr/data/15043554/fileData...,None,None,None,N,None,None,15043554,None
21008,24827,2,공공데이터,과학기술,안산도시공사_무인발권기 현황,안산도시공사 시설에 자동으로 입장권을 발권 해 주는 장치 현황 입니다.,None,None,None,None,...,[File],[https://www.data.go.kr/data/15066571/fileData...,None,None,None,N,None,None,15066571,None


In [4]:
df2 = df.iloc[: , 4]
df2 = df2.apply(lambda x: x.split('_'))

group_list = []
for i in range(len(df2)) : 
     group_info = df2[i][0]
     group_list.append(group_info)    

      
group_list = set(group_list) # 중복을 날립니다. 
group_list = list(group_list)

group_list


['대구광역시 중구 영어도서관 도서목록',
 '국가과학기술연구회 소관 출연(연) 특허정보',
 '경상북도',
 '대전광역시 중구',
 '부산광역시 동구 가로등 현황',
 '한국인터넷진흥원',
 '울산광역시 울주군 재난대응용 배수펌프장 시설현황',
 '경상북도교육청 경상북도청송교육지원청',
 '충청남도교육청',
 '경상남도 의령군 건축사사무소 현황',
 '대구광역시 중구 맛집 현황',
 '경상북도 청송군 모범음식점 현황',
 '건설근로자공제회',
 '한국지역난방공사 지역난방 무상교육 운영현황',
 '인천광역시 미추홀구',
 '충청남도 보령시',
 '한국식품연구원 연구성과정보(논문)',
 '부산광역시 여성회관 운영프로그램 현황',
 '부산광역시 남구 문화재매매업소 현황',
 '경상북도 영주시',
 '부산광역시 동구',
 '광주광역시 북구',
 '과학기술정보통신부 국립중앙과학관',
 '광주광역시',
 '경기도 오산시',
 '주택도시보증공사 보유 주식 현황 (출자전환주식)',
 '부산광역시 남구 진단용방사선발생장치현황',
 '부산광역시 남구 담배소매인지정 현황',
 '제주에너지공사',
 '경상북도교육청 경상북도교육청상주도서관',
 '부산광역시교육청 학원 및 교습소 현황',
 '광주광역시도시공사',
 '서울특별시 마포구',
 '원주시시설관리공단',
 '국가과학기술연구회 소관 출연(연) 융합연구과제별 과제수 및 사업금액 비율 현황',
 '인천광역시계양구시설관리공단 대형폐기물 처리 정보',
 '인천광역시 외국인 직접투자 현황(업종별)',
 '서울특별시 은평구 버스승차대 현황',
 '부산광역시교육청 부산광역시미래교육원',
 '대구도시철도공사와이파이설치현황',
 '부산광역시 남구 화물운송주선사업(이사화물)업체 현황',
 '한국보건복지인재원',
 '전라남도 영광군',
 '대구광역시교육청 대구광역시립수성도서관',
 '경상남도 사천시 제조업 현황',
 '충북대학교',
 '인천광역시교육청 인천광역시동부교육지원청',
 '경상북도교육청 경상북도교육청문화원',
 '한국법제연구원 연속간행물

In [ ]:
# 'on'가 아닌 ' on ' 양쪽 공백이 있어야 함 , 그렇지 않으면 boond 이런단어가 b_d 이런식으로 잘림
delete_words = [ ' of ' , ' by ' , ' and ' , ' on ' ,' at ' , ' for ' , ' about ' , ' amount ' , ' that ' , ' the ' 
                , ' with ' , ' as ' ,' an ' ]

with open("C:/Users/yoonsub/dict_data.pkl", 'rb') as f:
     dict_refine_words = pickle.load(f)

In [ ]:
for group in group_list :  
    en_translate = translator.translate(group, dest='en')
    logical_eng = en_translate.text
 
    
    logical_eng = logical_eng.strip()  # 양쪽 공백제거
    logical_eng = logical_eng.lower()  # 대문자 -> 소문자 변경
    logical_eng = logical_eng.replace('-', '')  # seo-gu ->seogu 변경

    # 중복제거
    logical_eng = logical_eng.split(' ')
    logical_eng = list(OrderedDict.fromkeys(logical_eng))
    logical_eng = " ".join(logical_eng)

    # 특정단어 혹은 전치사 삭제
    for word in delete_words:
        logical_eng = logical_eng.replace(word, " ")


    logical_eng = re.sub('[-=+,&#/\?:^.@*\"※~ㆍ!』‘|\(\)\[\]`\'…》\”\“\’·;_{}]', '', logical_eng) # 정규식 사용 특수문자 제거
    logical_eng = logical_eng.strip()  # 양쪽 공백제거
    time.sleep(0.25)

    # 회사명 약어 변경
    for j in dict_refine_words:
        logical_eng = logical_eng.replace(j, dict_refine_words[j])

    # 글자수 SQL 제약 확인

    #logical_eng = logical_eng.upper()  # 대문자로 모두 변경 , 테이블명이 50글자 이상이면 다시 쪼개서 제목을 줄임
    if len(logical_eng) >= 50:
        logical_eng = logical_eng.split(' ')[1:9]
        logical_eng = " ".join(logical_eng)

        if len(logical_eng) >= 50:
            logical_eng = logical_eng.split(' ')[2:8]
            logical_eng = " ".join(logical_eng)


    else:
        pass


    # 공백을 언더바로 변환 전처리

    logical_eng = logical_eng.replace('    ', '')
    logical_eng = logical_eng.strip()  # 양쪽 공백제거
    logical_eng = logical_eng.replace('   ', '_')  # 공백을 언더바로 변환
    logical_eng = logical_eng.strip()  # 양쪽 공백제거
    logical_eng = logical_eng.replace('  ', '_')  # 공백을 언더바로 변환
    logical_eng = logical_eng.strip()  # 양쪽 공백제거
    logical_eng = logical_eng.replace(' ', '_')  # 공백을 언더바로 변환


   
    print(group, ":" , logical_eng)
    
    
    
    
    
    
    

In [ ]:
logical_english_name = df['LOGICAL_TABLE_ENGLISH']
print(type(logical_english_name))

In [ ]:
from nltk.corpus import stopwords
from nltk.corpus import stopwords 
from nltk.tokenize import word_tokenize 

print(stopwords.words('english'))

In [ ]:
logical_english_name[0]

In [ ]:
words = [] 
for en in logical_english_name :     
    en = en.split("_")    
    en = " ".join(en)
    with open("words.txt" , 'a' , encoding ='utf8') as f : 
        f.write(en)  
    
    

In [ ]:
with open("words.txt" , 'r' , encoding ='utf8') as f : 
    text = f.read()

wordList = text.split()
print(wordList)

In [ ]:
wordCount = {} 
 
for word in wordList:
 
    # Get 명령어를 통해, Dictionary에 Key가 없으면 0리턴
 
    wordCount[word] = wordCount.get(word, 0) + 1 
    
    keys = sorted(wordCount.keys())
 
for word in keys:
 
    print(word + ':' + str(wordCount[word])) 